In [1]:
!pip install aif360
!pip install fairlearn
!pip install matplotlib seaborn scikit-learn pandas numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 54.2 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3


In [2]:
# First, let's make sure all packages are installed
try:
    import aif360
except ImportError:
    print("Installing AIF360 toolkit...")
    !pip install aif360
    import aif360

try:
    import fairlearn
except ImportError:
    print("Installing fairlearn...")
    !pip install fairlearn
    import fairlearn

# Now import all required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import Reweighing
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

print("All packages successfully imported! 🎉")

pip install 'aif360[inFairness]'


All packages successfully imported! 🎉


In [3]:
import urllib.request
import os

# Download the COMPAS dataset
dataset_url = "https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv"
dataset_path = "compas-scores-two-years.csv"

if not os.path.exists(dataset_path):
    print("Downloading COMPAS dataset...")
    urllib.request.urlretrieve(dataset_url, dataset_path)
    print("Dataset downloaded successfully!")
else:
    print("Dataset already exists!")

Dataset downloaded successfully!


In [4]:
# COMPAS Bias Analysis - Complete Working Version
def load_and_analyze_compas():
    # Load the dataset
    df = pd.read_csv('compas-scores-two-years.csv')

    # Data cleaning as done in ProPublica's analysis
    df = df[(df['days_b_screening_arrest'] <= 30) &
            (df['days_b_screening_arrest'] >= -30)]
    df = df[df['is_recid'] != -1]
    df = df[df['c_charge_degree'] != 'O']
    df = df[df['score_text'] != 'N/A']

    # Create binary race variable (African-American vs Caucasian)
    df['race_binary'] = df['race'].apply(lambda x: 1 if x == 'Caucasian' else 0)

    # Create binary labels
    df['two_year_recid'] = df['two_year_recid'].astype(int)

    return df

def analyze_bias_with_aif360(df):
    # Create BinaryLabelDataset for AIF360
    dataset = BinaryLabelDataset(
        df=df,
        label_names=['two_year_recid'],
        protected_attribute_names=['race_binary'],
        favorable_label=0,  # No recidivism
        unfavorable_label=1  # Recidivism occurred
    )

    # Calculate bias metrics
    privileged_groups = [{'race_binary': 1}]  # Caucasian
    unprivileged_groups = [{'race_binary': 0}]  # African-American

    metric = BinaryLabelDatasetMetric(
        dataset,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups
    )

    print("=== BIAS METRICS ANALYSIS ===")
    print(f"Statistical Parity Difference: {metric.statistical_parity_difference():.4f}")
    print(f"Disparate Impact Ratio: {metric.disparate_impact():.4f}")
    print(f"Consistency Score: {metric.consistency():.4f}")

    return metric

def create_bias_visualizations(df):
    # Calculate metrics for visualization
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 12))

    # Plot 1: False Positive Rates by Race
    fp_rates = []
    fn_rates = []
    races = ['African-American', 'Caucasian']

    for race in ['African-American', 'Caucasian']:
        race_data = df[df['race'] == race]

        # False Positive Rate
        fp = len(race_data[(race_data['score_text'] == 'High') &
                          (race_data['two_year_recid'] == 0)]) / len(race_data[race_data['two_year_recid'] == 0])
        fp_rates.append(fp)

        # False Negative Rate
        fn = len(race_data[(race_data['score_text'] == 'Low') &
                          (race_data['two_year_recid'] == 1)]) / len(race_data[race_data['two_year_recid'] == 1])
        fn_rates.append(fn)

    # Plot 1: False Positive Rates
    bars1 = ax1.bar(races, fp_rates, color=['#e74c3c', '#3498db'], alpha=0.7)
    ax1.set_title('False Positive Rates by Race\n(Predicted High Risk but No Recidivism)', fontsize=12, fontweight='bold')
    ax1.set_ylabel('False Positive Rate')
    ax1.grid(axis='y', alpha=0.3)

    # Add value labels on bars
    for bar in bars1:
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom')

    # Plot 2: False Negative Rates
    bars2 = ax2.bar(races, fn_rates, color=['#e74c3c', '#3498db'], alpha=0.7)
    ax2.set_title('False Negative Rates by Race\n(Predicted Low Risk but Did Reoffend)', fontsize=12, fontweight='bold')
    ax2.set_ylabel('False Negative Rate')
    ax2.grid(axis='y', alpha=0.3)

    for bar in bars2:
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.3f}', ha='center', va='bottom')

    # Plot 3: Score Distribution
    score_dist = pd.crosstab(df['race'], df['score_text'], normalize='index')
    score_dist.plot(kind='bar', ax=ax3, color=['#2ecc71', '#f1c40f', '#e74c3c'])
    ax3.set_title('Risk Score Distribution by Race', fontweight='bold')
    ax3.set_ylabel('Proportion')
    ax3.legend(title='Risk Score', bbox_to_anchor=(1.05, 1), loc='upper left')
    ax3.tick_params(axis='x', rotation=45)

    # Plot 4: Actual Recidivism vs Predicted
    actual_vs_predicted = pd.crosstab(df['race'],
                                    [df['score_text'], df['two_year_recid']],
                                    normalize='index')
    actual_vs_predicted.plot(kind='bar', ax=ax4, figsize=(12, 6))
    ax4.set_title('Actual Recidivism vs Risk Score by Race', fontweight='bold')
    ax4.set_ylabel('Proportion')
    ax4.legend(title='Risk Score / Recidivism', bbox_to_anchor=(1.05, 1), loc='upper left')
    ax4.tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.savefig('compas_bias_analysis_detailed.png', dpi=300, bbox_inches='tight')
    plt.show()

    return fp_rates, fn_rates

# Main execution
try:
    print("Starting COMPAS Bias Analysis...")
    df = load_and_analyze_compas()
    print(f"Dataset loaded with {len(df)} records")

    metric = analyze_bias_with_aif360(df)
    fp_rates, fn_rates = create_bias_visualizations(df)

    print("\n=== KEY FINDINGS ===")
    print(f"False Positive Rate - African-American: {fp_rates[0]:.3f}")
    print(f"False Positive Rate - Caucasian: {fp_rates[1]:.3f}")
    print(f"Disparity in FPR: {fp_rates[0]/fp_rates[1]:.2f}x higher for African-Americans")

except Exception as e:
    print(f"Error occurred: {e}")
    print("Make sure all packages are installed and dataset is available")

Starting COMPAS Bias Analysis...
Dataset loaded with 6172 records
Error occurred: Input DataFrames cannot contain NA values.
Make sure all packages are installed and dataset is available
